In [1]:
import pandas as pd
import json
import pymongo
import matplotlib.dates as mdates
from datetime import datetime, date
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random

In [2]:
def data_import(include_mums=False):
    """Connects to database and creates dataframe containing all columns. Drops unneeded columns and sets timestamp
     datatype. Correct any incorrect time values, sets data times and sorts"""

    collection_list=['Times']
    
    if include_mums:
        collection_list.append('Mumsnet_Times')
    all_records = []

    # Connects to db and gets collection
    db = get_db()

    for collection in collection_list:
        records =list(db[collection].find())
        all_records.append(records)
    
    #Flattens list
    all_records = [val for sublist in all_records for val in sublist]
    
    df=pd.DataFrame(all_records)
    
    # Makes column to indicate which database times are from
    non_mums = ['Harvey Williams', 'Sazzle', 'Leah', 'Tom', 'Joe', 'George Sheen', 'Oliver Folkard'] 
    df['mum'] = np.where(df['user'].isin(non_mums), False, True)
    
    return df

In [3]:
def get_db(write=False):
    if write:
        connection_string = "admin_connection_string"

    else:
        connection_string = "connection_string"

    try:
        with open("local/pass.json") as file:
            file = json.loads(file.read())
            connection_string = file.get(connection_string)
            client = pymongo.MongoClient(
                connection_string)
            db = client["PlusWord"]
            return db
    except Exception as e:
        print(e)

In [4]:
def format_for_streamlit(df):
    """Makes df more readable, converts times into plottable numbers and sets index"""

    df = df[['load_ts', 'time', 'user']]
    df['time'] = df['time'].str.replace(r'(^\d\d:\d\d$)', r'00:\1', regex=True)
    df['load_ts'] = pd.to_datetime(df['load_ts'], format='%Y-%m-%d %H:%M:%S.%f')
    # df['user'] = df['user'].astype('category')
    df = df.sort_values(by=['load_ts'])
    df = df.rename(columns={'load_ts': 'timestamp'})
    df['time_delta'] = pd.to_timedelta(df['time'].astype('timedelta64[ns]'))
    df['time_delta_as_num'] = time_delta_to_num(pd.to_timedelta(df['time'].astype('string')))
    df['sub_time_delta_as_num'] = time_delta_to_num(pd.to_timedelta(df['timestamp'].dt.time.astype('string')))


    df = df.set_index('timestamp')
    df = df.sort_index(ascending=False)

    return df

In [5]:
def format_for_streamlit(df):
    """Makes df more readable, converts times into plottable numbers and sets index"""

    df = df[['load_ts', 'time', 'user', 'mum']]
    df['time'] = df['time'].str.replace(r'(^\d\d:\d\d$)', r'00:\1', regex=True)
    df['load_ts'] = pd.to_datetime(df['load_ts'], format='%Y-%m-%d %H:%M:%S.%f')
    # df['user'] = df['user'].astype('category')
    df = df.sort_values(by=['load_ts'])
    df = df.rename(columns={'load_ts': 'timestamp'})
    df['time_delta'] = pd.to_timedelta(df['time'].astype('timedelta64[ns]'))
    df['time_delta_as_num'] = time_delta_to_num(pd.to_timedelta(df['time'].astype('string')))
    df['sub_time_delta_as_num'] = time_delta_to_num(pd.to_timedelta(df['timestamp'].dt.time.astype('string')))


    df = df.set_index('timestamp')
    df = df.sort_index(ascending=False)

    return df


In [6]:
def time_delta_to_num(time_delta):
    """ Takes in time delta and converts it into a number for plotting"""

    # specify a date to use for the times

    zero_date = datetime(2022, 6, 20)

    zero_num = mdates.date2num(zero_date)

    # adds zero_data to timedelta to convert

    time_delta_plus_date = [zero_date + time_unit for time_unit in time_delta]

    # convert datetimes to numbers

    time_delta_as_num = [mins - zero_num for mins in mdates.date2num(time_delta_plus_date)]

    return time_delta_as_num

In [7]:
df=data_import(include_mums=True)
df = format_for_streamlit(df)

/tmp/ipykernel_11124/1743044916.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = df['time'].str.replace(r'(^\d\d:\d\d$)', r'00:\1', regex=True)
/tmp/ipykernel_11124/1743044916.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['load_ts'] = pd.to_datetime(df['load_ts'], format='%Y-%m-%d %H:%M:%S.%f')


In [8]:
df

,time,user,mum,time_delta,time_delta_as_num,sub_time_delta_as_num
timestamp,,,,,,
2023-06-27 15:53:00.000,00:00:44,Floralnomad,True,0 days 00:00:44,0.000509,0.661806
2023-06-27 12:16:07.688,00:00:52,Oliver Folkard,False,0 days 00:00:52,0.000602,0.511200
2023-06-27 12:12:32.942,00:00:41,George Sheen,False,0 days 00:00:41,0.000475,0.508715
2023-06-27 09:53:55.976,00:00:21,Joe,False,0 days 00:00:21,0.000243,0.412453
2023-06-27 07:31:00.000,00:00:30,Albaniarocks,True,0 days 00:00:30,0.000347,0.313194
...,...,...,...,...,...,...
2022-06-04 09:45:00.000,00:36:59,Joe,False,0 days 00:36:59,0.025683,0.406250
2022-06-04 00:24:00.000,00:13:16,Harvey Williams,False,0 days 00:13:16,0.009213,0.016667
2022-06-03 13:29:00.000,00:14:48,Harvey Williams,False,0 days 00:14:48,0.010278,0.561806


In [9]:
df_difficulty = df.copy()

df_difficulty['date'] = df_difficulty.index.date

more_than_3_entries = df_difficulty['date'].value_counts() > 3

df_difficulty['mum'] = df_difficulty['mum'].astype('int64')

df_difficulty = df_difficulty.groupby([['date', 'mum']])['time_delta_as_num'].mean()

# df_difficulty[more_than_3_entries]

df_difficulty = df_difficulty.reset_index()

df_difficulty = df_difficulty.sort_values(by='time_delta_as_num', ascending=True)

# Selects 20 hardest



df_difficulty['time'] = mdates.num2timedelta(df_difficulty['time_delta_as_num'])

fig, ax = plt.subplots(figsize=(10, 5))

fig = sns.scatterplot(data=df_difficulty,
                      x='date',
                      y='time_delta_as_num',
                      hue='mum')

ax.yaxis_date()

ax.yaxis.set_major_formatter(mdates.DateFormatter("%M:%S"))

ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))

ax.set_xlabel(None)

ax.set_ylabel('Mean Time /mins')

ax.set_ylim(ymin=0)

# Formats df

df_difficulty = time_delta_as_num_to_time(df_difficulty)

df_difficulty = df_difficulty[['date', 'Time']]
df_difficulty = df_difficulty.set_index('date')

KeyError: 'mum'

In [ ]:
df

In [ ]:
df

In [ ]:
df.user.value_counts()

In [ ]:
less_than_5_entries = df['user'].value_counts() > 5


In [ ]:
less_than_5_entries

In [ ]:
df = df.groupby(df["user"])["time_delta_as_num"].max()
df

In [ ]:
df = df[less_than_5_entries]
df

In [ ]:
df = df.reset_index()

df = df.sort_values(by='time_delta_as_num', ascending=False)
df

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

fig = sns.barplot(data=df,
                  x="user",
                  y="time_delta_as_num",
                  ).set(
    ylabel='Time /mins',
    xlabel=None)

ax.yaxis_date()

ax.yaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))

plt.show()

In [ ]:
pal = sns.color_palette("hls", 20)

In [ ]:
pal